## 在"批踢踢實業坊(PTT)"網站抓股市看板上的新聞（爬連續多個頁面）
### <p style="line-height:1.6em;" align="justify;">本範例欲爬的目標網站：[[批踢踢實業坊網站]](https://www.ptt.cc/bbs/Stock/index.html)</p>

**※ 事前準備，安裝 pyquery 套件**

**在 Anaconda 的命令列（即 Anaconda Prompt 的選項，<font color=red>但是請務必以管理員身分執行</font>）中輸入指令 "<font color=blue>conda install -c anaconda pyquery</font>" 或是 "<font color=blue>pip install pyquery</font>"**

#### <p style="line-height:1.6em;" align="justify;">PyQuery 是一個類似於 jQuery 的解析網頁工具，使用 lxml 操作 xml 和 html 文檔，它的語法和 jQuery 很像。和 XPATH 以及 Beautiful Soup 比起來，PyQuery 更加靈活，提供增加節點的 class 信息，移除某個節點，提取文本信息等功能。<br><br>PyQuery 的用法 [[1]](https://www.twblogs.net/a/5c0aba23bd9eee6fb2138dd8) [[2]](https://pythonhosted.org/pyquery/) [[3 重要]](https://pyquery.readthedocs.io/_/downloads/en/latest/pdf/) [[4]](https://kknews.cc/zh-tw/code/6qqlppp.html) [[5*]](https://www.cnblogs.com/chenyangqit/p/16594908.html)</p>

### 以下在 Conda 上 install PyQuery 的方式二選一 [[1]](https://gist.github.com/yassineAlouini/bf1a4597ee4ae85f5925ecc39e01e4a4) [[官網]](https://anaconda.org/anaconda/pyquery)

In [ ]:
# Install Method 1 : 
# Install a pip package in the current Jupyter Kernel

import sys

!{sys.executable} -m pip install pyquery

In [ ]:
# Install Method 2 : 
# Install a conda package in the current Jupyter Kernel

#import sys
#!conda install  --yes --prefix {sys.prefix} pyquery 

!conda install -c anaconda pyquery

In [ ]:
# 可以去掉 Python 輸出時，因為軟體版本所引起的警告的警告。

import warnings

warnings.filterwarnings('ignore')

In [ ]:
from pyquery import PyQuery as pq

from collections import OrderedDict

import pandas as pd

In [ ]:
#
# 開使爬蟲動作
#
import requests

url = 'https://www.ptt.cc/bbs/Stock/index.html'   # 到最新的頁面(此為預設)

#url = 'https://www.ptt.cc/bbs/Stock/index8814.html' # 到特定頁面

q = pq(requests.get(url).text)     # requests.get() 方法不可以省

In [ ]:
#
# 經過解析後的 q 就是網站的 html DOM Tree
#
print(q)

In [ ]:
#
# 獲取本頁的頁碼，在這裡使用正規表達式
#

import re

#page_no = q('.wide:nth-child(2)')
page_no = q('div.btn-group.btn-group-paging')

rule = r'href="\/bbs\/Stock\/index([0-9]+).html".+(.)頁'

results = re.findall(rule, str(page_no))

print(results)

In [ ]:
if len(results) == 2:
    
    # 
    # 此情況為所在頁面不是 "最舊頁面" 也不是 " 最新頁面，所以上下頁面均有頁號資訊
    # 
    current_page_no = int(results[0][0]) + 1     # 設定現在所在頁面的號碼

elif len(results) == 1 and results[0][1] == '下':

    # 
    # 此情況為所在頁面是 "最舊頁面" ，所以只有 "下" 頁面有頁號資訊
    #    
    current_page_no = int(results[0][0]) - 1     # 設定現在所在頁面的號碼

elif len(results) == 1 and results[0][1] == '上':

    # 
    # 此情況為所在頁面是 "最新頁面" ，所以只有 "上" 頁面有頁號資訊
    #    
    current_page_no = int(results[0][0]) + 1    # 設定現在所在頁面的號碼
    
print('現在頁面：%d' % current_page_no)

In [ ]:
i = current_page_no

q_list = []

print()

total_pages = int(input(f'請輸入要抓取的頁數(從第 {current_page_no} 頁往舊的頁面)：'))    # 設定抓前面 10 頁的文章資訊

print()

#
# 從最新的頁面往前(舊的頁面)抓 total_pages 頁
#
for i in range(0, total_pages):
    
    url = 'https://www.ptt.cc/bbs/Stock/index' + str(current_page_no - i) + '.html'
    
    print(url)
         
    q_list.append( pq(requests.get(url).text) )

In [ ]:
#
# 重要訣竅：直接使用 print() 方法可列印出目標 URL 網頁內的頁面原始碼
#
print(q_list[0])    # 試著列印出 q_list 中第一頁的內容

In [ ]:
#
# 將每一頁的內容列印在螢幕上呈現，每一頁之間以雙虛線隔開
#

count = 0

for q in q_list:
    
    count += 1
    
    print('\npage', count, '===============================================================\n')
    print(q)
    print('\npage', count, '===============================================================\n')

###  <font color=blue>未完，接下來可參考 "批踢踢實業坊_Stock_處理單頁版" 的範例來處理每頁的後續動作</font>